<a href="https://colab.research.google.com/github/anskong/ai_ref_from_lessons/blob/main/LGCNS_LangChain_005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  도구를 이용한 대화모델과의 상호작용


In [ ]:
# 1) 필수 패키지 설치 (LangChain 0.3.0 이상)
!pip install --quiet --upgrade "langchain-openai>=0.3.0" "langchain-core>=0.3.0" openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 34.3 MB/s eta 0:00:00


In [ ]:
# 2) API 키 로드 (Colab의 Tools → Secrets에 등록된 값 사용)
import os
from google.colab import userdata

# API 키 등록
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 3) 필요한 모듈 임포트
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# 4) LLM 인스턴스 생성
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4.1-nano"  # 필요 시 모델 변경
)

# 5) 초기 메시지 구성
messages = [
    SystemMessage("당신의 이름은 하루입니다."),
    HumanMessage("내 이름은 데이브입니다."),
    AIMessage("안녕하세요. 데이브"),
    HumanMessage("하루, 포드의 라이트를 20도 왼쪽으로 돌려줘."),
]

# 6) Tool 정의 및 바인딩
@tool
def light_control(degrees: float) -> bool:
    """
    조명을 오른쪽으로 degrees 만큼 회전시킵니다.
    실제 하드웨어 대신 콘솔에 출력합니다.
    """
    print(f"[Tool 실행] 조명을 {degrees}도 회전시킴")
    return True

llm_with_tool = llm.bind_tools([light_control])

# 7) 첫 번째 모델 응답
response = llm_with_tool.invoke(messages)
messages.append(response)

# 8) 툴 호출 처리
if response.tool_calls:
    for call in response.tool_calls:
        value = light_control.invoke(call["args"])
        messages.append(ToolMessage(content=value, tool_call_id=call["id"]))

# 9) 후속 응답
final_response = llm_with_tool.invoke(messages)

# 10) 최종 출력
print("=== 모델 최종 응답 ===")
print(final_response.content)

[Tool 실행] 조명을 20.0도 회전시킴
=== 모델 최종 응답 ===
포드의 라이트를 20도 왼쪽으로 돌렸어요.
